# Exercici 1

In [1]:
RANDOM_STATE = 0
TEST_SIZE = 0.3

In [2]:
import numpy as np
from tensorflow import keras
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

## Apartat 1

En aquest apartat es demana, utilitzant una funció incorporada en la llibreria `Sklearn`, carregar el dataset de les cares de Olivetti. Un cop s'ha carregat correctament, s'ha de fer la distinció entre dades de entrenament i dades de test, les quals ens serviran per valorar, de forma objectiva el funcionament del model. A continuació, es decideix, per tal de millorar el funcionament dels models, escalar les dades entre els valors \[0, 1\]. En l'enunciat es comenta que per tal d'escalar les dades s'han de dividir tots els valors entre el màxim absolut del conjunt. En canvi, s'ha cregut oportu utilitzar la funcionalitat de sklearn `MinMaxScaler`, la qual s'auto descriu com un transformador de valors entre un rang determinat.

In [22]:
X, y = fetch_olivetti_faces(return_X_y=True)

In [23]:
scaler = MinMaxScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [24]:
enc = OneHotEncoder().fit(y.reshape(-1, 1))
y_encoded = enc.transform(y.reshape(-1, 1)).toarray()

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=TEST_SIZE, random_state=RANDOM_STATE)

## Apartat 2

In [26]:
def make_pca_with_n_components(n_components, X_train, X_test):
  pca = PCA(n_components=n_components).fit(X_train)
  return pca.transform(X_train), pca.transform(X_test)

In [27]:
def create_custom_model_v2(input_size, inter_neurons, output_size):
  model = keras.Sequential()
  model.add(keras.Input(shape=input_size))
  model.add(keras.layers.Dense(inter_neurons, activation='relu'))
  model.add(keras.layers.Dense(output_size, activation='softmax'))
  return model

In [28]:
def train_and_test_model(model, X_train, X_test, y_train, y_test, batch_size, epochs):
  model.compile(optimizer=keras.optimizers.SGD(), loss=keras.losses.CategoricalCrossentropy())
  model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,verbose=False)
  predictions_train = model.predict(X_train, verbose=False)
  predictions_test = model.predict(X_test, verbose=False)
  true_predictions_train = sum([1 if np.argmax(pred) == np.argmax(real) else 0 for pred, real in zip(predictions_train, y_train)])
  true_predictions_test = sum([1 if np.argmax(pred) == np.argmax(real) else 0 for pred, real in zip(predictions_test, y_test)])
  return (true_predictions_train, len(y_train)), (true_predictions_test, len(y_test))

In [29]:
testing_pca_values = [10, 20]
testing_internal_neurons = [25, 50, 75, 100]

In [30]:
for pca in testing_pca_values:
  for in_neurons in testing_internal_neurons:
    X_train_pca, X_test_pca = make_pca_with_n_components(pca, X_train, X_test)
    model = create_custom_model_v2(pca, in_neurons, len(y_train[0]))
    train, test = train_and_test_model(model, X_train_pca, X_test_pca, y_train, y_test, 16, 200)
    print('By using', pca, 'components and',in_neurons, 'intermediate neurons. We have achieved the following results.',
          'Our model is able to guess the', round(train[0]/train[1], 2), '% in training and', round(test[0]/test[1], 2), '% in validation.')

By using 10 components and 25 intermediate neurons. We have achieved the following results. Our model is able to guess the 0.99 % in training and 0.76 % in validation.
By using 10 components and 50 intermediate neurons. We have achieved the following results. Our model is able to guess the 1.0 % in training and 0.83 % in validation.
By using 10 components and 75 intermediate neurons. We have achieved the following results. Our model is able to guess the 1.0 % in training and 0.84 % in validation.
By using 10 components and 100 intermediate neurons. We have achieved the following results. Our model is able to guess the 1.0 % in training and 0.84 % in validation.
By using 20 components and 25 intermediate neurons. We have achieved the following results. Our model is able to guess the 1.0 % in training and 0.83 % in validation.
By using 20 components and 50 intermediate neurons. We have achieved the following results. Our model is able to guess the 1.0 % in training and 0.83 % in validati

# Apartat 3

In [31]:
def create_custom_model_v3(inter_neurons):
  model = keras.Sequential()
  model.add(keras.Input(shape=(64,64,1,)))
  model.add(keras.layers.Conv2D(filters=inter_neurons, kernel_size=3, strides=1, activation="relu"))
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(40, activation="softmax"))
  return model

In [32]:
reshaped_x = X_scaled.reshape(-1,64,64,1)

In [33]:
X_train_3, X_test_3, y_train, y_test = train_test_split(reshaped_x, y_encoded, test_size=TEST_SIZE, random_state=RANDOM_STATE)

In [34]:
testing_internal_neurons = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [35]:
for in_neurons in testing_internal_neurons:
  model = create_custom_model_v3(in_neurons)
  train, test = train_and_test_model(model, X_train_3, X_test_3, y_train, y_test, 16, 200)
  print('By using', in_neurons, 'intermediate neurons. We have achieved the following results.',
        'Our model is able to guess the', round(train[0]/train[1], 2), '% in training and', round(test[0]/test[1], 2), '% in validation.')

By using 1 intermediate neurons. We have achieved the following results. Our model is able to guess the 1.0 % in training and 0.93 % in validation.
By using 2 intermediate neurons. We have achieved the following results. Our model is able to guess the 1.0 % in training and 0.93 % in validation.
By using 3 intermediate neurons. We have achieved the following results. Our model is able to guess the 1.0 % in training and 0.94 % in validation.
By using 4 intermediate neurons. We have achieved the following results. Our model is able to guess the 1.0 % in training and 0.93 % in validation.
By using 5 intermediate neurons. We have achieved the following results. Our model is able to guess the 1.0 % in training and 0.92 % in validation.
By using 6 intermediate neurons. We have achieved the following results. Our model is able to guess the 1.0 % in training and 0.93 % in validation.
By using 7 intermediate neurons. We have achieved the following results. Our model is able to guess the 1.0 % in

# Apartat 4

In [40]:
model_2 = create_custom_model_v2(20, 100, len(y_train[0]))
model_2.summary()

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_51 (Dense)            (None, 100)               2100      
                                                                 
 dense_52 (Dense)            (None, 40)                4040      
                                                                 
Total params: 6,140
Trainable params: 6,140
Non-trainable params: 0
_________________________________________________________________


In [41]:
model_3 = create_custom_model_v3(9)
model_3.summary()

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 62, 62, 9)         90        
                                                                 
 flatten_11 (Flatten)        (None, 34596)             0         
                                                                 
 dense_53 (Dense)            (None, 40)                1383880   
                                                                 
Total params: 1,383,970
Trainable params: 1,383,970
Non-trainable params: 0
_________________________________________________________________


Para calcular el número de parámetros de un modelo de inteligencia artificial con capas convolucionales, primero debes conocer el tamaño de la entrada, el tamaño del kernel o filtro y el número de canales de la imagen de entrada. Además, debes conocer el número de filtros o kernels que se utilizan en cada capa y el tamaño del stride o paso utilizado por el filtro.

El número de parámetros de una capa convolucional se puede calcular utilizando la siguiente fórmula:

(tamaño del kernel * tamaño del kernel * número de canales de entrada + 1) * número de filtros

Si tienes varias capas convolucionales en tu modelo, debes sumar el número de parámetros de cada una de ellas para obtener el número total de parámetros del modelo.

Por otra parte, para calcular el número de parámetros de un modelo de inteligencia artificial con capas densas o completamente conectadas, debes conocer el tamaño de la entrada y el número de neuronas o unidades en cada capa.

El número de parámetros de una capa densa se puede calcular utilizando la siguiente fórmula:

(tamaño de entrada + 1) * número de neuronas

Tanto en las capas convolucionales como las densas, el "1" se suma para tener en cuenta el término de sesgo, que es un parámetro adicional que se utiliza en cada uno de los filtros

Un modelo de inteligencia artificial con un alto número de parámetros puede tener algunas ventajas y desventajas en comparación con un modelo con un número más bajo de parámetros. Algunas de las principales ventajas y desventajas se describen a continuación:

- (+) Mayor capacidad de aprendizaje
- (+) Mayor capacidad de adaptación
- (-) Mayor tiempo de entrenamiento
- (-) Mayor riesgo de sobreajuste

En general, es importante encontrar un equilibrio adecuado entre el número de parámetros y el rendimiento del modelo. A menudo es útil utilizar técnicas como la validación cruzada y el ajuste de hiperparámetros para encontrar el número óptimo de parámetros para un modelo específico.

Si no se hubiese cambiado el tamaño de los datos de entrada, en el modelo de capas densas hubiese habido (4096 + 1) * 100 = 409700 parámetros. En cambio, en el modelo convolucional, no se ha reducido el tamaño de entrada y, por lo tanto, tendríamos el mismo número de parámetros.